<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout,Conv2D,AvgPool2D,Flatten,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.losses import MeanAbsoluteError,MeanSquaredError
#import tensorflow_addons as tfa
#from tf.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
#from tfa.optimizers import AdamW
#from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.applications.resnet import ResNet50
import numpy as np
import pandas as pd


def load_train(path):
    #ResNet50
    #print(path)
    lables=pd.read_csv(path+'labels.csv')

    dir=path+'final_files/'


    datagen = ImageDataGenerator(validation_split=0.25,
                                 rescale=1./255,
                                 rotation_range=60)

    train_data = datagen.flow_from_dataframe(dataframe=lables,
                                             directory='/datasets/faces/final_files/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    subset='training',
    x_col='file_name',
    y_col='real_age',
    seed=12345)

    return train_data

def load_test(path):
    #ResNet50
    #print(path)
    lables=pd.read_csv(path+'labels.csv')
    datagen = ImageDataGenerator(validation_split=0.25,rescale=1./255
    )
    test_data = datagen.flow_from_dataframe(dataframe=lables,
        directory='/datasets/faces/final_files/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    x_col='file_name',
    y_col='real_age',
    subset='validation',
    seed=12345)

    

    return test_data


def create_model(input_shape):
    #Lenet
    
    mae=MeanAbsoluteError(reduction="auto", name="mean_absolute_percentage_error")
    mse=MeanSquaredError()
    adam=Adam(learning_rate=0.0001)
    backbone = ResNet50(input_shape=(224, 224, 3),
                    weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                    include_top=False)
    #backbone.trainable = False

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())

    model.add(Dropout(0.25))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(1,activation='relu'))
    model.compile(loss=mse, optimizer=adam, metrics=['mae'])
    model.summary()

    '''def scheduler(epoch, lr):

      if epoch < 5:
        return lr
      else:
        return lr * 0.3'''
    

    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=15,
                steps_per_epoch=None, validation_steps=None):
    #callback = LearningRateScheduler(scheduler)
    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)
    return model